In [177]:
import pandas as pd
import numpy as np 
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from datetime import datetime
import os
import random 
import seaborn as sns
import functools
from fbprophet import Prophet
from statistics import mean
from sklearn.metrics import r2_score

In [178]:
data = pd.read_csv(os.path.expanduser("~/Desktop/DS_Competition/ucsbdata.csv")).dropna(axis=0)
data['Index']=pd.to_datetime(data['Index'])
data = data.drop(['OPEN', 'HIGH', 'LOW'], axis=1) 

data_2018 = data[499:-251] #1/02/2009 - 12/29/2017    
data = data[499:] #1/02/2009 - 12/31/2018 
data_2018.tail()

,Index,R,US10YR,EPS,PER,CLOSE,BDIY,VIX,PCR,MVOLE,...,MYSTERYVAR3,TOM,HULBERTSENTIMENT,BUBBLE,CASH,DEBT,DIVIDEND,RETURN,STOCKS,WAR
16607,2017-12-22,-0.001248,2.4810,108.43,24.747972,2683.34,1366.0,9.90,0.64,18850.14,...,10.979090,0,61.07,63.739178,162.922123,46.603509,76.494006,37.076353,54.537971,33.839705
16608,2017-12-26,0.000431,2.4756,108.43,24.719929,2680.50,1366.0,10.25,0.58,16934.40,...,10.972110,1,56.31,63.739178,160.336057,37.043815,71.394405,34.028981,54.537971,32.335718
16609,2017-12-27,0.002001,2.4107,108.43,24.739268,2682.62,1366.0,10.47,0.55,15027.73,...,10.966183,1,55.24,71.617054,131.889338,31.069006,55.075684,42.155305,47.468234,35.343692
16610,2017-12-28,-0.003831,2.4305,108.43,24.784619,2687.54,1366.0,10.18,0.57,18068.49,...,10.960120,1,55.79,65.887690,137.061468,42.421143,74.454166,41.647410,59.587783,33.839705
16611,2017-12-29,0.007078,2.4054,108.43,24.670409,2673.61,1366.0,11.04,0.59,17940.39,...,10.942710,1,55.79,61.590666,137.061468,39.433739,79.553766,40.631619,58.577820,33.839705


In [179]:
df_prophet=data_2018[['Index','R']]
df_prophet=df_prophet.sort_values('Index')
df_prophet=df_prophet.rename(columns={'Index':'ds','R':'y'})

df_prophet.head(3)

,ds,y
14347,2009-01-02,-0.001187
14348,2009-01-05,0.006650
14349,2009-01-06,-0.022717


In [180]:
m=Prophet()
m.fit(df_prophet)


future=m.make_future_dataframe(periods=365)

future.tail()
forecast=m.predict(future)
forecast.head(5)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2009-01-02,0.000392,-0.011083,0.011578,0.000392,0.000392,-0.000176,-0.000176,-0.000176,0.000234,0.000234,0.000234,-0.000410,-0.000410,-0.000410,0.0,0.0,0.0,0.000216
1,2009-01-05,0.000391,-0.012112,0.010693,0.000391,0.000391,-0.000656,-0.000656,-0.000656,0.000353,0.000353,0.000353,-0.001009,-0.001009,-0.001009,0.0,0.0,0.0,-0.000265
2,2009-01-06,0.000391,-0.012762,0.011914,0.000391,0.000391,-0.000732,-0.000732,-0.000732,0.000473,0.000473,0.000473,-0.001204,-0.001204,-0.001204,0.0,0.0,0.0,-0.000340
3,2009-01-07,0.000391,-0.011594,0.011208,0.000391,0.000391,-0.000893,-0.000893,-0.000893,0.000498,0.000498,0.000498,-0.001391,-0.001391,-0.001391,0.0,0.0,0.0,-0.000502
4,2009-01-08,0.000391,-0.013824,0.010227,0.000391,0.000391,-0.001542,-0.001542,-0.001542,0.000021,0.000021,0.000021,-0.001563,-0.001563,-0.001563,0.0,0.0,0.0,-0.001151


# Check that Predicted R-Average for 2018 is similar to actual R-average (0.0008706)

In [197]:
results = forecast.drop(['trend', 'yhat_lower', 'yhat_upper','trend_lower', 'trend_upper', 'additive_terms', 'additive_terms_lower',
                        'additive_terms_upper', 'weekly','weekly_lower','weekly_upper','yearly','yearly_lower','yearly_upper',
                        'multiplicative_terms','multiplicative_terms_lower','multiplicative_terms_upper'], axis=1) 
results = results[:-1]


results.rename(columns = {'yhat':'PREDICTED'}, inplace = True) 
results.set_index('ds')

R_values = data[:-1]
R_values.drop(['US10YR','ASP', 'TERM', 'VRP', 
         'LOAN','DXY', 'HTIME', 'LTIME', 'IC', 
         'INF', 'UR', 'URR', 'HS', 'FOMC', 'PPIR', 
         'NAPMPMI', 'NAPMNEWO', 'NAPMPRIC', 'PTB', 'CATY','PCR', 'NOS', 'MVOLE', 'NYSEADV', 'RV',
         'RELINF', 'SIM', 'TOM', 'BDIY', 'SENTIMENT1',
         'SENTIMENT2', 'SENTIMENT3', 'SENTIMENT4', 'HULBERTSENTIMENT',
         'MYSTERYVAR1', 'MYSTERYVAR2','DEBT', 'RETURN', 'STOCKS', 'WAR','EPS','PER','CLOSE','VIX','ADVDECL', 'BAA',
              'BER', 'DEL', 'BBY', 'TYVIX', 'CRP', 'INDPRO', 'CAPE', 'FEDFUNDS', 'DVY', 'AAA', 'SI',
              'VVIX', 'US3M', 'MYSTERYVAR3', 'CASH', 'DIVIDEND', 'BUBBLE'], axis=1)

R_values=R_values[['Index','R']]
R_values=R_values.sort_values('Index')
R_values=R_values.rename(columns={'Index':'ds','R':'y'})
R_values.set_index('ds')



result = results.merge(R_values, on='ds', how='inner') 
result.rename(columns={'ds':'date','PREDICTED':'prediction', 'y':'target'}, inplace=True)
result.head(10)


,date,prediction,target
0,2009-01-02,0.000216,-0.001187
1,2009-01-05,-0.000265,0.006650
2,2009-01-06,-0.000340,-0.022717
3,2009-01-07,-0.000502,0.004069
4,2009-01-08,-0.001151,-0.021655
5,2009-01-09,-0.001093,-0.022731
6,2009-01-12,-0.001299,0.001834
7,2009-01-13,-0.001230,-0.022745
8,2009-01-14,-0.001226,0.000352
9,2009-01-15,-0.001690,0.007785


In [196]:
print("Average Predicted R value for 2018:", mean(result['prediction'])) 
print("Average Actual R Value for 2018:", mean(result['target']))

R2 = r2_score(result['target'], result['prediction'])
print("R^2 Score:", R2)

Average Predicted R value for 2018: 0.000598748534729625
Average Actual R Value for 2018: 0.000534048394494265
R^2 Score: 0.006041763992634341


 # Check that Predicted R-Average for 2019 is > -0.016

In [114]:
stock_price_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
stock_price = df_prophet
df = pd.merge(stock_price, stock_price_forecast, on='ds', how='right')

# Stock Trading Simulation

In [14]:
stock_price['dayname'] = stock_price['ds'].dt.day_name()
stock_price['month'] = stock_price['ds'].dt.month
stock_price['year'] = stock_price['ds'].dt.year
stock_price['month/year'] = stock_price['month'].map(str) + '/' + stock_price['year'].map(str) 

stock_price = pd.merge(stock_price, 
                       stock_price['month/year'].drop_duplicates().reset_index(drop=True).reset_index(),
                       on='month/year',
                       how='left')

stock_price = stock_price.rename(columns={'index':'month/year_index'})
stock_price.head()

,ds,y,dayname,month,year,month/year,month/year_index
0,2007-01-05,1409.71,Friday,1,2007,1/2007,0
1,2007-01-08,1412.84,Monday,1,2007,1/2007,0
2,2007-01-09,1412.11,Tuesday,1,2007,1/2007,0
3,2007-01-10,1414.85,Wednesday,1,2007,1/2007,0
4,2007-01-11,1423.82,Thursday,1,2007,1/2007,0


Hold = 1,734
Prophet = 1,858
Prophet Thresh = 1,399
Seasonality = 1,953
Optimized Prophet Thresh = 1,663